<center>
<img src="https://image.aladin.co.kr/product/12609/37/cover500/k372532974_1.jpg" width="200" height="200"><br>
</center>

# 1장 서울시 구별 cctv 현황 분석

서울시의 구별 CCTV 현황을 분석해보자. 단순히 어디에 CCTV가 많이 설치 됐는지부터 시작해서 구별 인구 대비 비율을 확인하는 것까지 진행 한다. 특히 인구 현황을 보면서 구별 인구 현황에 대해서도 확인해보자. 그리고 구별 CCTV 현황을 시각화하는 부분도 알아보자.저자의 코드와, 데이터의 원본은 다음을 확인하자. 

 - Github: [PinkWink](https://github.com/PinkWink/DataScience)

### 1-1 파이썬에서 텍스트 파일과 엑셀 파일 읽기

pandas 모듈을 사용해 read_csv 함수를 사용하자. CCTV 데이터는 UTF-8로 인코딩 되어 있어서 옵션값으로 encoding = 'utf-8'을 지정 해주었다. 그리고 이는 CCTV_Seoul로 지정해주자. 

In [135]:
import pandas as pd
CCTV_Seoul = pd.read_csv("data/01. CCTV_in_Seoul.csv", encoding = 'utf-8')
CCTV_Seoul.head()

,기관명,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,2780,1292,430,584,932
1,강동구,773,379,99,155,377
2,강북구,748,369,120,138,204
3,강서구,884,388,258,184,81
4,관악구,1496,846,260,390,613


CCTV_Seoul의 컬럼명을 알아보자. 

In [136]:
CCTV_Seoul.columns

Index(['기관명', '소계', '2013년도 이전', '2014년', '2015년', '2016년'], dtype='object')

이렇게 첫번째 컬럼의 기관명을 '구별'로 변경 해주자. 다음의 코드를 확인하자. 컬럼명을 변경하기 위해서는 rename 메서드를 사용한다. 그리고 옵션으로 inplace=True옵션을 사용해 변수의 내용을 갱신한다.

In [137]:
CCTV_Seoul.rename(columns={CCTV_Seoul.columns[0] : '구별'}, inplace = True)
CCTV_Seoul.head()

,구별,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,2780,1292,430,584,932
1,강동구,773,379,99,155,377
2,강북구,748,369,120,138,204
3,강서구,884,388,258,184,81
4,관악구,1496,846,260,390,613


이제 두 번째 받은 엑셀 파일을 읽어오자.

In [138]:
pop_Seoul = pd.read_csv("data/01. population_in_Seoul.csv", encoding='utf-8')
pop_Seoul.head()

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,세대당인구,65세이상고령자
0,기간,자치구,세대,합계,합계,합계,한국인,한국인,한국인,등록외국인,등록외국인,등록외국인,세대당인구,65세이상고령자
1,기간,자치구,세대,계,남자,여자,계,남자,여자,계,남자,여자,세대당인구,65세이상고령자
2,2017.1/4,합계,4202888,10197604,5000005,5197599,9926968,4871560,5055408,270636,128445,142191,2,1321458
3,2017.1/4,종로구,72654,162820,79675,83145,153589,75611,77978,9231,4064,5167,2,25425
4,2017.1/4,중구,59481,133240,65790,67450,124312,61656,62656,8928,4134,4794,2,20764


데이터가 처음에 엑셀로 되어 있어서 csv로 변환 시켜주었다. 그러나 이는 보니 1~3행이 알수가 없어서 다음의 옵션을 주자.

In [139]:
pop_Seoul = pd.read_csv("data/01. population_in_Seoul.csv", 
                        header = 2,
                        encoding='utf-8')
pop_Seoul.head()

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2017.1/4,합계,4202888,10197604,5000005,5197599,9926968,4871560,5055408,270636,128445,142191,2.0,1321458
1,2017.1/4,종로구,72654,162820,79675,83145,153589,75611,77978,9231,4064,5167,2.0,25425
2,2017.1/4,중구,59481,133240,65790,67450,124312,61656,62656,8928,4134,4794,2.0,20764
3,2017.1/4,용산구,106544,244203,119132,125071,229456,111167,118289,14747,7965,6782,2.0,36231
4,2017.1/4,성동구,130868,311244,153768,157476,303380,150076,153304,7864,3692,4172,2.0,39997


In [140]:
pop_Seoul = pop_Seoul.iloc[:, [1,3,6, 9,13]]


In [141]:
pop_Seoul.head()

,자치구,계,계.1,계.2,65세이상고령자
0,합계,10197604,9926968,270636,1321458
1,종로구,162820,153589,9231,25425
2,중구,133240,124312,8928,20764
3,용산구,244203,229456,14747,36231
4,성동구,311244,303380,7864,39997


이렇게 필요한 데이터만 수집을 해두었으니 컬럼명을 변경 해주자.

In [142]:
pop_Seoul.rename(columns={pop_Seoul.columns[0] : '구별',
                          pop_Seoul.columns[1] : '인구수',
                          pop_Seoul.columns[2] : '한국인',
                          pop_Seoul.columns[3] : '외국인',
                          pop_Seoul.columns[4] : '고령자'}, inplace=True)

pop_Seoul.head()

,구별,인구수,한국인,외국인,고령자
0,합계,10197604,9926968,270636,1321458
1,종로구,162820,153589,9231,25425
2,중구,133240,124312,8928,20764
3,용산구,244203,229456,14747,36231
4,성동구,311244,303380,7864,39997


### 1-2 pandas 이용해서 CCTV와 인구 현황 데이터 파악

이제 pandas를 이용해서 두 데이터를 정리해보자. 우선 CCTV 데이터를 먼저 파악하자.

#### 1-2-1 서울시 CCTV 데이터 파악

In [143]:
import pandas as pd
import numpy as np

In [144]:
CCTV_Seoul.head()

,구별,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,2780,1292,430,584,932
1,강동구,773,379,99,155,377
2,강북구,748,369,120,138,204
3,강서구,884,388,258,184,81
4,관악구,1496,846,260,390,613


구별 CCTV 데이터에서 CCTV 전체 개수인 소계로 정렬을 해보자. 이때 ascending=TRUE옵션을 주어 오름차순으로 정렬해준다. 다음을 보면 CCTV 전체 개수가 가장 작은 구는 도봉수, 마포구, 송파구, 중랑구 의 순서로 나타나는 것을 알수 있다. 이때 의아한 점이라면 송파구의 개수가 서울에서 하위 3개 인데 송파구는 이는 집어 두자. 

In [145]:
CCTV_Seoul.sort_values(by = '소계', ascending = True).head()

,구별,소계,2013년도 이전,2014년,2015년,2016년
9,도봉구,485,238,159,42,386
12,마포구,574,314,118,169,379
17,송파구,618,529,21,68,463
24,중랑구,660,509,121,177,109
23,중구,671,413,190,72,348


다음은 ascending=False옵션으로 내림차순을 해준 결과 이다. 가장 많은 구는 강남구, 양천구, 서초구, 은평구, 용산구로 나타난다. 

In [146]:
CCTV_Seoul.sort_values(by = '소계', ascending = False).head()

,구별,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,2780,1292,430,584,932
18,양천구,2034,1843,142,30,467
14,서초구,1930,1406,157,336,398
21,은평구,1873,1138,224,278,468
20,용산구,1624,1368,218,112,398


또한 2014년부터 2016년까지 3년간 CCTV 수를 더하고 2013년 이전 CCTV수로 나누어 3년간의 CCTV 증가율을 계산하자. 그리고 '최근증가율'이라는 새로운 컬럼을 생성해주자.

In [147]:
CCTV_Seoul['최근증가율'] = (CCTV_Seoul['2016년'] + CCTV_Seoul['2015년'] + CCTV_Seoul['2014년']) / CCTV_Seoul['2013년도 이전'] * 100

CCTV_Seoul.sort_values(by = '최근증가율', ascending = False).head()

,구별,소계,2013년도 이전,2014년,2015년,2016년,최근증가율
22,종로구,1002,464,314,211,630,248.922414
9,도봉구,485,238,159,42,386,246.638655
12,마포구,574,314,118,169,379,212.101911
8,노원구,1265,542,57,451,516,188.929889
1,강동구,773,379,99,155,377,166.490765


이렇게 최근 증가율을 확인해보면 종로구, 도봉구, 마포구 순서로 증가 했음을 알 수 있다. 다음은 서울시의 인구 현황을 살펴 보자.

#### 1-2-2 서울시 인구 현황

In [148]:
pop_Seoul.head()

,구별,인구수,한국인,외국인,고령자
0,합계,10197604,9926968,270636,1321458
1,종로구,162820,153589,9231,25425
2,중구,133240,124312,8928,20764
3,용산구,244203,229456,14747,36231
4,성동구,311244,303380,7864,39997


이를 보니 0번행에 합계가 보인다. 아마 서울시 전체 합계를 넣어둔 것인데 이는 필요 없으니 drop 메서드를 사용해 지우도록 하자. 

In [149]:
pop_Seoul.drop([0], inplace=True)
pop_Seoul.head()

,구별,인구수,한국인,외국인,고령자
1,종로구,162820,153589,9231,25425
2,중구,133240,124312,8928,20764
3,용산구,244203,229456,14747,36231
4,성동구,311244,303380,7864,39997
5,광진구,372164,357211,14953,42214


pop_Seoul 데이터의 '구별'컬럼의 고육값을 확인해보자. 

In [150]:
pop_Seoul['구별'].unique()

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'], dtype=object)

이렇게 파악해보니 끝에 nan이 있으니 이를 한번 확인해보자. isnull 메서드로 확인할 수 있다. 26번 행이 NaN이 있음을 확인할 수 있다.

In [151]:
pop_Seoul[pop_Seoul['구별'].isnull()]

,구별,인구수,한국인,외국인,고령자


이제 각 구별 전체 인구를 이용해서 구별 '외국인비율'과 '고령자비율'을 계산해보자.

In [152]:
pop_Seoul['외국인비율'] = pop_Seoul['외국인'] / pop_Seoul['인구수'] * 100
pop_Seoul['고령자비율'] = pop_Seoul['고령자'] / pop_Seoul['인구수'] * 100
pop_Seoul.head()

,구별,인구수,한국인,외국인,고령자,외국인비율,고령자비율
1,종로구,162820,153589,9231,25425,5.669451,15.615404
2,중구,133240,124312,8928,20764,6.700690,15.583909
3,용산구,244203,229456,14747,36231,6.038828,14.836427
4,성동구,311244,303380,7864,39997,2.526635,12.850689
5,광진구,372164,357211,14953,42214,4.017852,11.342849


인구수로 정렬하여 다음을 확인해보자. 

In [153]:
pop_Seoul.sort_values(by='인구수', ascending = False).head()

,구별,인구수,한국인,외국인,고령자,외국인비율,고령자비율
24,송파구,667483,660584,6899,72506,1.033584,10.862599
16,강서구,603772,597248,6524,72548,1.080540,12.015794
23,강남구,570500,565550,4950,63167,0.867660,11.072217
11,노원구,569384,565565,3819,71941,0.670725,12.634883
21,관악구,525515,507203,18312,68082,3.484582,12.955291


송파구, 강서구, 강남구, 노원구, 관악구 순서로 인구가 많다는걸 확인할 수 있다.

In [155]:
pop_Seoul.sort_values(by='외국인', ascending = False).head()

,구별,인구수,한국인,외국인,고령자,외국인비율,고령자비율
19,영등포구,402985,368072,34913,52413,8.663598,13.006191
17,구로구,447874,416487,31387,56833,7.007998,12.689506
18,금천구,255082,236353,18729,32970,7.342345,12.925255
21,관악구,525515,507203,18312,68082,3.484582,12.955291
6,동대문구,369496,354079,15417,54173,4.172440,14.661322


외국인 숫자의 비율은 영등포, 구로구, 금천구, 관악구, 동대문구 순으로 높은것을 알 수 있다. 

In [156]:
pop_Seoul.sort_values(by='외국인비율', ascending = False).head()

,구별,인구수,한국인,외국인,고령자,외국인비율,고령자비율
19,영등포구,402985,368072,34913,52413,8.663598,13.006191
18,금천구,255082,236353,18729,32970,7.342345,12.925255
17,구로구,447874,416487,31387,56833,7.007998,12.689506
2,중구,133240,124312,8928,20764,6.700690,15.583909
3,용산구,244203,229456,14747,36231,6.038828,14.836427


이번에는 외국인 비율을 알아보자. 외국인 비율은 영등포, 금천구, 구로구, 중구, 용산구 순서로 나타나는 것을 확인할 수 있다.

In [157]:
pop_Seoul.sort_values(by='고령자', ascending = False).head()

,구별,인구수,한국인,외국인,고령자,외국인비율,고령자비율
16,강서구,603772,597248,6524,72548,1.080540,12.015794
24,송파구,667483,660584,6899,72506,1.033584,10.862599
12,은평구,494388,489943,4445,72334,0.899091,14.631019
11,노원구,569384,565565,3819,71941,0.670725,12.634883
21,관악구,525515,507203,18312,68082,3.484582,12.955291


고령자 순서로 보면 강서구, 송파구, 은평구, 노원구, 관악구임을 알 수 있다.

In [158]:
pop_Seoul.sort_values(by='고령자비율', ascending = False).head()

,구별,인구수,한국인,외국인,고령자,외국인비율,고령자비율
9,강북구,330192,326686,3506,54813,1.061806,16.600342
1,종로구,162820,153589,9231,25425,5.669451,15.615404
2,중구,133240,124312,8928,20764,6.700690,15.583909
3,용산구,244203,229456,14747,36231,6.038828,14.836427
13,서대문구,327163,314982,12181,48161,3.723221,14.720797


고령자의 비율을 보면 강북구, 종로구, 중구, 용산구, 서대문구 순서임을 확인할 수 있다.

지금까지 간략하게나마 CCTV의 현황을 파악해보았으나 아직은 부족하다. 인구대비 CCTV 현황을 알기 위해 두 데이터를 병합해주자.

#### 1-3 CCTV 데이터와 인구 현황 데이터 합치고 분석

판다스의 merge 메서드를 사용해서 두 데이터를 합쳐 주자. 공통 컬럼인 '구별'컬럼을 기준으로 합쳐 주면 된다.

In [162]:
data_result = pd.merge(CCTV_Seoul, pop_Seoul, on='구별')
data_result.head()

,구별,소계,2013년도 이전,2014년,2015년,2016년,최근증가율,인구수,한국인,외국인,고령자,외국인비율,고령자비율
0,강남구,2780,1292,430,584,932,150.619195,570500,565550,4950,63167,0.867660,11.072217
1,강동구,773,379,99,155,377,166.490765,453233,449019,4214,54622,0.929765,12.051638
2,강북구,748,369,120,138,204,125.203252,330192,326686,3506,54813,1.061806,16.600342
3,강서구,884,388,258,184,81,134.793814,603772,597248,6524,72548,1.080540,12.015794
4,관악구,1496,846,260,390,613,149.290780,525515,507203,18312,68082,3.484582,12.955291


이제 의미 없는 컬럼은 모두 제거 해주자. 열을 삭제하기 위해서든 del 명령어를 사용한다. 

In [163]:
del data_result['2013년도 이전']
del data_result['2014년']
del data_result['2015년']
del data_result['2016년']
data_result.head()

,구별,소계,최근증가율,인구수,한국인,외국인,고령자,외국인비율,고령자비율
0,강남구,2780,150.619195,570500,565550,4950,63167,0.867660,11.072217
1,강동구,773,166.490765,453233,449019,4214,54622,0.929765,12.051638
2,강북구,748,125.203252,330192,326686,3506,54813,1.061806,16.600342
3,강서구,884,134.793814,603772,597248,6524,72548,1.080540,12.015794
4,관악구,1496,149.290780,525515,507203,18312,68082,3.484582,12.955291
